<div style="text-align:center; font-size:2.5em; color:#4B0055; letter-spacing:2px; background-color:#F5F5F5; padding:20px; border-radius:10px;">
    Modelo e Análise Exploratória de Dados: 
    <span style="color:#FFA726; font-weight:bold;">Fluxar</span>
    <div style="text-align:center; margin-top:10px;">
        <img src="logo_fluxar.png" alt="Logo Fluxar" style="height:70px;">
    </div>
</div>


<div style="background:#F7F3FF; border-left:8px solid #4B0055; padding:16px; margin-top:20px;">
    <span style="font-size:1.3em; color:#4B0055; font-weight:bold;">Modelo escolhido:</span><br>
    <span style="color:#FFA726; font-size:1.1em;">Forecasting & Stockout Prediction — <b>Fluxar</b></span>
</div>


<div style="background:linear-gradient(90deg, #ffc164ff 0%, #FF1493 100%); color:white; padding:18px; border-radius:10px; margin-top:20px;">
    <span style="font-size:1.2em; font-weight:bold;">Objetivos Concretos do Modelo do Fluxar:</span>
    <ul style="margin-top:10px;">
        <li>Prever o número de dias até o estoque acabar (<i>days_to_stockout</i>) para cada SKU, com janela de previsão de 14 dias.</li>
        <li>Assim, o gestor consegue se antecipar antes que o produto falte.</li>
        <li>Reduzir a ocorrência de <b>stockouts</b> em pelo menos <span>20%</span> (meta inicial), evitando perdas de vendas e desperdício logístico.</li>
        <li>Gerar alertas automáticos no app (push notification ou no painel web) quando o estoque projetado cair abaixo de um limite crítico.</li>
        <li>Fornecer insights de tendência (Exemplo: quais SKUs mais frequentemente entram em risco de stockout).</li>
    </ul>
</div>

### <span style="color:#4B0055; font-family:'Segoe UI', 'Arial', sans-serif;">Importação de Bibliotecas</span>

In [43]:
# Para os pip install:
# !pip install xgboost lightgbm catboost pandas numpy scikit-learn joblib

In [44]:
# Para o dataset 
import os
import shutil
import kagglehub

# Para análise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para modelagem
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
import joblib

# Para modelos
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

# Para treinamento dos modelos
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
import time



### <span style="color:#4B0055; font-family:'Segoe UI', 'Arial', sans-serif;">Construção de pipeline de criação de ML </span>

<!-- Etapa 1 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">1. Definição do Problema</span>
  <p style="color:#4B0055;">
    Objetivo: prever o número de dias até o estoque acabar (<em>days_to_stockout</em>) para cada SKU/loja, com janela de previsão de 14 dias.<br>
    Tipo de problema: regressão, pois a saída é uma variável contínua.<br>
    Metas concretas: reduzir stockouts, gerar alertas automáticos, antecipar reposição.
  </p>
</div>

In [45]:
print("Etapa 1 concluída: Problema definido")

Etapa 1 concluída: Problema definido


<!-- Etapa 2 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">2. Coleta de Dados</span>
  <p style="color:#4B0055;">
    Dataset: <em>Retail Store Inventory Forecasting</em> (Kaggle)<br>
    Inclui dados históricos diários de estoque, vendas, promoções, feriados, clima, preços e categorias de produto.<br>
    Objetivo: garantir histórico suficiente por SKU e loja para treinar modelos robustos.
  </p>
</div>

### <span style="color:#4B0055; font-family:'Segoe UI', 'Arial', sans-serif;">Carregar dados </span>
Dataset proveniente do Kaggle

In [46]:
# Caminho final desejado
destino = r"C:\Users\mayumishimizu-ieg\OneDrive - Instituto Germinare\Área de Trabalho\2º ANO\Interdisciplinar\Matemática Aplicada"
destino_dataset = os.path.join(destino, "dataset_fluxar")
os.makedirs(destino_dataset, exist_ok=True)

# Baixar dataset do Kaggle
try:
    path = kagglehub.dataset_download("anirudhchauhan/retail-store-inventory-forecasting-dataset")
    print("✅ | Dataset baixado em:", path)
except Exception as e:
    print("❌ | Erro ao baixar dataset:", e)

# Copiar e renomear o primeiro CSV encontrado para "dataset_modelo.csv"
for file in os.listdir(path):
    full_file_path = os.path.join(path, file)
    if os.path.isfile(full_file_path) and file.endswith(".csv"):
        destino_final = os.path.join(destino_dataset, "dataset_modelo.csv")
        shutil.copy(full_file_path, destino_final)
        print(f"📂 | Arquivo salvo e renomeado como: {destino_final}")
        break  # garante que só salva o primeiro CSV

# Verificação rápida
df = pd.read_csv(os.path.join(destino_dataset, "dataset_modelo.csv"))
print(f"\n🔎 | dataset_modelo.csv carregado com sucesso!")
print(f"Dimensão: {df.shape}")
print(df.head())

✅ | Dataset baixado em: C:\Users\mayumishimizu-ieg\.cache\kagglehub\datasets\anirudhchauhan\retail-store-inventory-forecasting-dataset\versions\1
📂 | Arquivo salvo e renomeado como: C:\Users\mayumishimizu-ieg\OneDrive - Instituto Germinare\Área de Trabalho\2º ANO\Interdisciplinar\Matemática Aplicada\dataset_fluxar\dataset_modelo.csv

🔎 | dataset_modelo.csv carregado com sucesso!
Dimensão: (73100, 15)
         Date Store ID Product ID     Category Region  Inventory Level  \
0  2022-01-01     S001      P0001    Groceries  North              231   
1  2022-01-01     S001      P0002         Toys  South              204   
2  2022-01-01     S001      P0003         Toys   West              102   
3  2022-01-01     S001      P0004         Toys  North              469   
4  2022-01-01     S001      P0005  Electronics   East              166   

   Units Sold  Units Ordered  Demand Forecast  Price  Discount  \
0         127             55           135.47  33.50        20   
1         150      

<!-- Etapa 3 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">3. Pré-processamento de Dados</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Padronizar nomes de colunas: <code>Inventory Level → Inventory_Level</code></li>
    <li>Tratar valores ausentes e duplicados</li>
    <li>Criar variável alvo: <code>days_to_stockout = Inventory_Level / (Units_Sold + 1e-5)</code></li>
    <li>Criar features temporais: dia da semana, mês, fim de semana, feriado</li>
    <li>Transformar variáveis categóricas em numéricas (Label Encoding ou One-Hot)</li>
    <li>Remover outliers e dividir dados em treino/teste</li>
  </ul>
</div>

In [47]:
# Carregar dataset
caminho = r"C:\Users\mayumishimizu-ieg\OneDrive - Instituto Germinare\Área de Trabalho\2º ANO\Interdisciplinar\Matemática Aplicada\dataset_fluxar\dataset_modelo.csv"
df = pd.read_csv(caminho)

# Padronizar nomes de colunas
df = df.rename(columns=lambda x: x.strip().replace(" ", "_").replace("/", "_"))

# Converter datas
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])  # remove linhas com datas inválidas

# Ordenar dados
df = df.sort_values(by=["Product_ID", "Store_ID", "Date"])

# Criar variável alvo com média móvel de 7 dias
df['Units_Sold_Rolling7'] = df.groupby(['Product_ID', 'Store_ID'])['Units_Sold'].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)
df['days_to_stockout'] = df['Inventory_Level'] / (df['Units_Sold_Rolling7'] + 1e-5)

# Features temporais
df['dayofweek'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

# Tratamento de valores nulos (mais seguro que drop direto)
for col in df.select_dtypes(include='number'):
    df[col] = df[col].fillna(df[col].median())
for col in df.select_dtypes(include='object'):
    df[col] = df[col].fillna("desconhecido")

# Tratamento de outliers usando IQR
for col in ['Inventory_Level', 'Units_Sold', 'days_to_stockout']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    df = df[(df[col] >= lim_inf) & (df[col] <= lim_sup)]

# Encoding das variáveis categóricas
categorical_cols = ['Category', 'Region', 'Weather_Condition', 'Holiday_Promotion']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Checar se restou string e tratar
cat_cols_remaining = df.select_dtypes(include='object').columns.tolist()
if cat_cols_remaining:
    df = pd.get_dummies(df, columns=cat_cols_remaining, drop_first=True)

print("✅ Pré-processamento concluído. Shape final:", df.shape)


✅ Pré-processamento concluído. Shape final: (70701, 50)


<!-- Etapa 4 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">4. Escolha dos Modelos</span>
  <p style="color:#4B0055;">
    Três modelos escolhidos para treinar em paralelo:<br>
    1. <b>Regressão Linear</b> – baseline, fácil de interpretar<br>
    2. <b>Random Forest Regressor</b> – captura relações não-lineares, fornece importância das features<br>
    3. <b>XGBoost Regressor</b> – boosting eficiente, alta performance em dados tabulares
  </p>
</div>

<div style="background:#ffcee5ff; border-left:8px solid #f81e84ff; padding:18px; border-radius:10px; margin-top:24px;">
    <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">Modelos escolhidos, levando em conta interpretação, performance e aplicabilidade:</span>
    <ul style="margin-top:12px;">
        <li>
            <span style="color:#f81e84ff; font-weight:bold;">Regressão Linear / Ridge Regression</span><br>
            <span style="color:#4B0055;">Boa para prever <b>days_to_stockout</b> como uma variável contínua.<br>
            Fácil de explicar para professores/gestores.<br>
            Serve como baseline simples.</span>
        </li>
        <li style="margin-top:10px;">
            <span style="color:#f81e84ff; font-weight:bold;">Random Forest Regressor</span><br>
            <span style="color:#4B0055;">Modelo de árvore baseado em entropia/Gini, mas muito mais robusto.<br>
            Capta relações não-lineares entre variáveis (ex.: <span><b>sazonalidade</b></span>, promoções).<br>
            Permite medir a importância das features (quais colunas mais influenciam).</span>
        </li>
        <li style="margin-top:10px;">
            <span style="color:#f81e84ff; font-weight:bold;">XGBoost (Extreme Gradient Boosting)</span><br>
            <span style="color:#4B0055;">Modelo avançado, muito usado em competições Kaggle.<br>
            Ótimo para dados tabulares com comportamento complexo.<br>
            Normalmente supera Random Forest em performance.</span>
        </li>
        <li style="margin-top:10px;">
            <span style="color:#f81e84ff; font-weight:bold;">LightGBM Regressor</span><br>
            <span style="color:#4B0055;">Muito rápido em datasets grandes.<br>
            Funciona bem com features numéricas e categóricas.<br>
            Boosting eficiente, frequentemente entrega MAPE menor com menos tempo de treino.</span>
        </li>
        <li style="margin-top:10px;">
            <span style="color:#f81e84ff; font-weight:bold;">CatBoost Regressor</span><br>
            <span style="color:#4B0055;">Excelente para dados com muitas variáveis categóricas (ex.: Product_ID, Store_ID).<br>
            Não exige tanto pré-processamento (Label/One-Hot Encoding).<br>
            Pode reduzir erros percentuais (MAPE) e é fácil de implementar em produção.</span>
        </li>
    </ul>
</div>


In [48]:
print("Modelos definidos: Linear/Ridge, Random Forest, XGBoost, LightGBM, CatBoost")

Modelos definidos: Linear/Ridge, Random Forest, XGBoost, LightGBM, CatBoost


<!-- Etapa 5 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">5. Treinamento dos Modelos</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Usar TimeSeriesSplit ou divisão treino/teste</li>
    <li>Treinar os modelos em paralelo para cada batch de dados novo</li>
    <li>Ajustar parâmetros iniciais básicos para evitar underfitting (n_estimators, learning_rate, max_depth etc.)</li>
  </ul>
</div>


<!-- Etapa 5b -->
<div style="background:#ffe8c6ff; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">Treinamento Inicial dos Modelos</span>
  <p style="color:#4B0055;">
    Nesta etapa, treinamos todos os modelos escolhidos com um número menor de estimadores/iterações para observar rapidamente qual modelo apresenta a <b>menor MAPE</b> e melhor desempenho com informações básicas.
    <br><br>
    Objetivo: identificar os modelos mais promissores antes de fazer um treinamento completo com mais dados e hiperparâmetros ajustados.
  </p>
</div>


In [49]:
# Separar features e target
X = df.drop(columns=['days_to_stockout', 'Date', 'Units_Sold_Rolling7'])
y = df['days_to_stockout']

# Divisão treino/teste (80/20 sem embaralhar)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Inicializar modelos com parâmetros reduzidos para treino rápido
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "RandomForest": RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1),
    "XGBoost": XGBRegressor(n_estimators=50, learning_rate=0.05, random_state=42, n_jobs=-1),
    "LightGBM": lgb.LGBMRegressor(n_estimators=50, learning_rate=0.05, random_state=42, n_jobs=-1),
    "CatBoost": CatBoostRegressor(iterations=50, learning_rate=0.05, verbose=0, random_state=42)
}

# Função MAPE customizada para evitar divisão por zero
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-5))) * 100

# Treinar e avaliar
results = {}

for name, model in models.items():
    print(f"\n🚀 Treinando {name}...")
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()
    
    start_pred = time.time()
    y_pred = model.predict(X_test)
    end_pred = time.time()
    
    mape = mean_absolute_percentage_error_safe(y_test, y_pred)
    results[name] = {'MAPE': mape, 
                     'Treino_s': end_train - start_train, 
                     'Predicao_s': end_pred - start_pred}
    
    print(f"📊 {name} treinado com sucesso!")
    print(f"MAPE: {mape:.2f}% | Tempo de treino: {end_train - start_train:.2f}s | Tempo de predição: {end_pred - start_pred:.4f}s")
    
    # Importância das features, se disponível
    if hasattr(model, "feature_importances_"):
        feat_imp = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
        print("\n🔎 Principais features:")
        print(feat_imp.head(5))



🚀 Treinando LinearRegression...
📊 LinearRegression treinado com sucesso!
MAPE: 26.06% | Tempo de treino: 0.24s | Tempo de predição: 0.0211s

🚀 Treinando Ridge...
📊 Ridge treinado com sucesso!
MAPE: 26.06% | Tempo de treino: 0.12s | Tempo de predição: 0.0159s

🚀 Treinando RandomForest...
📊 RandomForest treinado com sucesso!
MAPE: 24.84% | Tempo de treino: 21.65s | Tempo de predição: 0.2650s

🔎 Principais features:
Inventory_Level    0.657811
Demand_Forecast    0.053911
Units_Sold         0.043724
Units_Ordered      0.036737
Price              0.028542
dtype: float64

🚀 Treinando XGBoost...
📊 XGBoost treinado com sucesso!
MAPE: 26.12% | Tempo de treino: 0.76s | Tempo de predição: 0.0396s

🔎 Principais features:
Inventory_Level       0.727011
Units_Sold            0.045301
Demand_Forecast       0.033116
Product_ID_P0005      0.007611
Category_Furniture    0.007107
dtype: float32

🚀 Treinando LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was

<!-- Etapa 5c -->
<div style="background:#ffe8c6ff; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">Treinamento Avançado dos Modelos Selecionados</span>
  <p style="color:#4B0055;">
    Com base nos resultados do treinamento inicial, selecionamos os <b>2 melhores modelos</b> (menor MAPE) para um treinamento mais completo usando <b>200 estimadores/iterações</b> e todos os dados disponíveis.
    <br><br>
    Objetivo: avaliar a performance final de cada modelo em condições ideais e decidir qual será o modelo principal para previsão de <b>days_to_stockout</b>.
  </p>
</div>


In [50]:
# Modelos selecionados para treino completo
best_models = {
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42),
    "LightGBM": lgb.LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
}

# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-5))) * 100  # evita divisão por zero

# Treinar e avaliar os 2 melhores modelos
for name, model in best_models.items():
    print(f"\n🚀 Treinando {name}...")
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()
    
    start_pred = time.time()
    y_pred = model.predict(X_test)
    end_pred = time.time()
    
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    print(f"📊 {name} treinado com sucesso!")
    print(f"MAPE: {mape:.2f}%")
    print(f"⏱️ Tempo de treino: {end_train - start_train:.2f} s | Tempo de predição: {end_pred - start_pred:.4f} s")
    
    # Exibir importância das features, se disponível
    if hasattr(model, "feature_importances_"):
        feat_imp = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
        print("\n🔎 Principais features:")
        print(feat_imp.head(5))



🚀 Treinando RandomForest...
📊 RandomForest treinado com sucesso!
MAPE: 24.67%
⏱️ Tempo de treino: 261.16 s | Tempo de predição: 1.7065 s

🔎 Principais features:
Inventory_Level       0.657697
Demand_Forecast       0.053558
Units_Sold            0.044217
Units_Ordered         0.036593
Competitor_Pricing    0.028606
dtype: float64

🚀 Treinando LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001103 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1547
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 43
[LightGBM] [Info] Start training from score 2.050650
📊 LightGBM treinado com sucesso!
MAPE: 24.57%
⏱️ Tempo de treino: 0.68 s | Tempo de predição: 0.0311 s

🔎 Principais features:
Inventory_Level    1348
Units_Sold          697
month               593
Demand_Forecast     584
Units_Ordered 

<!-- Etapa 6 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">6. Avaliação do Modelo</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Métricas principais: MAE, MAPE, R², MSE e RMSE</li>
    <li>Comparar resultados entre os modelos treinados</li>
    <li>Visualizar predição vs valores reais para identificar erros sistemáticos</li>
  </ul>
</div>

In [51]:
# Avaliação completa dos modelos
results = {}

for name, model in models.items():
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {'MAE': mae, 'MAPE': mape, 'MSE': mse, 'RMSE': rmse, 'R2': r2}
    
    print(f"\n📊 | {name} - MAE: {mae:.3f} | MAPE: {mape:.2f}% | MSE: {mse:.3f} | RMSE: {rmse:.3f} | R²: {r2:.3f}")



📊 | LinearRegression - MAE: 0.456 | MAPE: 26.06% | MSE: 0.361 | RMSE: 0.601 | R²: 0.664

📊 | Ridge - MAE: 0.456 | MAPE: 26.06% | MSE: 0.361 | RMSE: 0.601 | R²: 0.664

📊 | RandomForest - MAE: 0.458 | MAPE: 24.84% | MSE: 0.370 | RMSE: 0.608 | R²: 0.656

📊 | XGBoost - MAE: 0.457 | MAPE: 26.12% | MSE: 0.361 | RMSE: 0.600 | R²: 0.664

📊 | LightGBM - MAE: 0.457 | MAPE: 26.22% | MSE: 0.361 | RMSE: 0.600 | R²: 0.664

📊 | CatBoost - MAE: 0.461 | MAPE: 26.89% | MSE: 0.364 | RMSE: 0.603 | R²: 0.661


<!-- Etapa 7 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">7. Ajuste de Hiperparâmetros</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Aplicar Grid Search, Random Search ou Otimização Bayesiana para parâmetros como: n_estimators, max_depth, learning_rate, min_child_weight</li>
    <li>Melhorar performance e evitar overfitting/underfitting</li>
  </ul>
</div>

In [52]:
# # Não necessário, mas útil para salvar modelos treinados

# # Parâmetros para Random Forest
# param_grid_rf = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # GridSearch para Random Forest
# grid_rf = GridSearchCV(
#     estimator=RandomForestRegressor(random_state=42),
#     param_grid=param_grid_rf,
#     scoring='neg_mean_absolute_error',  # MAPE direto não é suportado no scikit-learn
#     cv=3,
#     n_jobs=-1,
#     verbose=2
# )
# grid_rf.fit(X_train, y_train)
# models['RandomForest'] = grid_rf.best_estimator_
# print("✅ Melhor RF:", grid_rf.best_params_)
# print("MAE do RF otimizado:", -grid_rf.best_score_)

# # Parâmetros para XGBoost
# param_grid_xgb = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.7, 0.9, 1.0],
#     'colsample_bytree': [0.7, 0.9, 1.0]
# }

# # GridSearch para XGBoost
# grid_xgb = GridSearchCV(
#     estimator=XGBRegressor(random_state=42),
#     param_grid=param_grid_xgb,
#     scoring='neg_mean_absolute_error',
#     cv=3,
#     n_jobs=-1,
#     verbose=2
# )
# grid_xgb.fit(X_train, y_train)
# models['XGBoost'] = grid_xgb.best_estimator_
# print("✅ Melhor XGBoost:", grid_xgb.best_params_)
# print("MAE do XGBoost otimizado:", -grid_xgb.best_score_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


KeyboardInterrupt: 

<!-- Etapa 8 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">8. Validação Cruzada</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Usar TimeSeriesSplit para avaliar estabilidade do modelo em diferentes períodos</li>
    <li>Garante que o modelo generaliza bem para dados futuros</li>
  </ul>
</div>

In [ ]:
# Também não é necessário, mas útil para salvar modelos treinados

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
cv_results = {}

for name, model in models.items():
    mae_scores, mape_scores, mse_scores, rmse_scores, r2_scores = [], [], [], [], []
    
    for train_index, test_index in tscv.split(X):
        X_tr, X_val = X.iloc[train_index], X.iloc[test_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        
        mae_scores.append(mean_absolute_error(y_val, y_pred))
        mape_scores.append(mean_absolute_percentage_error(y_val, y_pred))
        mse_scores.append(mean_squared_error(y_val, y_pred))
        rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        r2_scores.append(r2_score(y_val, y_pred))
    
    cv_results[name] = {
        'MAE_mean': np.mean(mae_scores),
        'MAPE_mean': np.mean(mape_scores),
        'MSE_mean': np.mean(mse_scores),
        'RMSE_mean': np.mean(rmse_scores),
        'R2_mean': np.mean(r2_scores)
    }
    
    print(f"\n📊 {name} - CV média | MAE: {np.mean(mae_scores):.2f}, MAPE: {np.mean(mape_scores):.2f}%, "
          f"MSE: {np.mean(mse_scores):.3f}, RMSE: {np.mean(rmse_scores):.3f}, R²: {np.mean(r2_scores):.2f}")



📊 LinearRegression - CV média | MAE: 0.47, MAPE: 26.95%, MSE: 0.379, RMSE: 0.615, R²: 0.65

📊 Ridge - CV média | MAE: 0.47, MAPE: 26.92%, MSE: 0.378, RMSE: 0.615, R²: 0.65


KeyboardInterrupt: 

<!-- Etapa 9 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">9. Seleção do Melhor Modelo</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Comparar métricas de todos os modelos treinados</li>
    <li>Selecionar o modelo com menor MAE/RMSE e boa interpretação</li>
    <li>Salvar modelo final para deploy</li>
  </ul>
</div>

In [ ]:
import joblib

melhor_modelo_nome = min(results, key=lambda x: results[x]['MAE'])
melhor_modelo = models[melhor_modelo_nome]
print("Melhor modelo selecionado:", melhor_modelo_nome)

# Salvar modelo
joblib.dump(melhor_modelo, f"best_model_{melhor_modelo_nome}.pkl")
print("Modelo salvo com sucesso.")


<!-- Etapa 10 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">10. Deploy / Implementação</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Integrar modelo ao app Fluxar para gerar alertas automáticos de stockout</li>
    <li>Pode ser via API, painel web ou notificação push</li>
  </ul>
</div>

In [ ]:
# Não fazemos esse passo no notebook.

<!-- Etapa 11 -->
<div style="background:#F7F3FF; border-left:8px solid #FFA726; padding:18px; border-radius:10px; margin-top:24px; margin-bottom:24px;">
  <span style="font-size:1.2em; color:#4B0055; font-weight:bold;">11. Monitoramento e Manutenção</span>
  <ul style="color:#4B0055; margin-top:12px;">
    <li>Acompanhar performance do modelo ao longo do tempo</li>
    <li>Re-treinar com novos dados ou ajustar hiperparâmetros em caso de concept drift</li>
  </ul>
</div>

In [ ]:
# Não fazemos esse passo no notebook.